In [69]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import MinMaxScaler
from time import time
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [6]:
x_train.shape
x_test.shape

(60000, 28, 28)

In [7]:
x_train = x_train.reshape((60000,-1))
print(x_train.shape)

x_test = x_test.reshape((10000,-1))
print(x_test.shape)


(60000, 784)
(10000, 784)


In [8]:
print(x_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [78]:
model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [79]:
name = ".\\logs"
logdir = os.path.join('logs')
batch_size = 32
es = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
tb = TensorBoard(log_dir=name,profile_batch = 100000000, histogram_freq=0, write_images=True)
callbacks = [es,tb]
model.fit(x_train,y_train,epochs=50, batch_size=batch_size, callbacks=callbacks)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 7s 122us/sample - loss: 0.2625 - acc: 0.9202
Epoch 2/50
60000/60000 [==============================] - 7s 114us/sample - loss: 0.1091 - acc: 0.9672
Epoch 3/50
60000/60000 [==============================] - 7s 123us/sample - loss: 0.0812 - acc: 0.9765
Epoch 4/50
60000/60000 [==============================] - 7s 117us/sample - loss: 0.0658 - acc: 0.9805
Epoch 5/50
60000/60000 [==============================] - 7s 116us/sample - loss: 0.0537 - acc: 0.9839
Epoch 6/50
60000/60000 [==============================] - 7s 116us/sample - loss: 0.0439 - acc: 0.9862
Epoch 7/50
60000/60000 [==============================] - 7s 114us/sample - loss: 0.0398 - acc: 0.9884
Epoch 8/50
60000/60000 [==============================] - 7s 111us/sample - loss: 0.0334 - acc: 0.9900
Epoch 9/50
60000/60000 [==============================] - 7s 113us/sample - loss: 0.0296 - acc: 0.9913
Epoch 10/50
60000/60000 [=========================

In [77]:
%load_ext tensorboard
%tensorboard --logdir logs/fit


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 1912.

In [48]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 1s 80us/sample - loss: 0.1064 - acc: 0.9742


[0.10635768901547417, 0.9742]

In [54]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred = nb.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.5544


In [58]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.9695


In [ ]:
aph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')